This file explores the integration between listenbrainz, musicbrainz, and spotify.

It's separate from the application logic because the approach requires exploration to optimize and understand.
There's a lot more edge cases integratiing multiple edge cases than I had to contend with in using only Spotify.

Below, I sort out those edge cases, explore likelihood of error cases, so I can encode these states in tests.

Import Libs

In [2]:
import os
import time
import json
import requests
from requests import HTTPError
from requests.auth import HTTPBasicAuth

Set up Script Variables

In [3]:
artists = ["Noah Gundersen", "Julien Baker", "Phoebe Bridgers"]

Load Environment Variables

In [4]:
listenbrainz_api_key = os.getenv("LISTENBRAINZ_API_KEY")
spotify_client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
spotify_client_id = os.getenv("SPOTIFY_CLIENT_ID")
# listenbrainz_api_key, spotify_client_secret, spotify_client_id

Provide useful context around request errors

In [5]:
def handle_error(response):
    try:
        response.raise_for_status()
    except HTTPError as error:
        error.add_note(f"Request: {error.request}")
        error.add_note(f"Request headers: {error.request.headers}")
        error.add_note(f"Request body: {error.request.body}")
        error.add_note(f"Response: {error.response}")
        error.add_note(f"Response headers: {error.response.headers}")
        error.add_note(f"Response body: {error.response.text}")
        raise

Get Spotify Auth Token

In [6]:
spotify_auth = requests.post(
    url="https://accounts.spotify.com/api/token",
    auth=HTTPBasicAuth(spotify_client_id, spotify_client_secret),
    data={"grant_type": "client_credentials"}
)
handle_error(spotify_auth)

auth_json = spotify_auth.json()
spotify_token = auth_json['access_token']
# spotify_token

Get playlist based on artists

In [7]:
lb_radio = requests.get(
    url="https://api.listenbrainz.org/1/explore/lb-radio",
    params={
        "mode": "easy",
        "prompt": " ".join([f'artist:({artist})' for artist in artists])
    },
    headers={"Authorization": f"Bearer {listenbrainz_api_key}"}
)
handle_error(lb_radio)

tracks = lb_radio.json()["payload"]["jspf"]["playlist"]["track"]
len(tracks), tracks[:2]

(50,
 [{'album': 'Far Cry 5 Presents: Into the Flames (Original Game Soundtrack)',
   'creator': 'Greg Holden',
   'duration': 154946,
   'extension': {'https://musicbrainz.org/doc/jspf#track': {'artist_identifiers': ['24a73f2f-f322-4f62-810d-60483c47878b'],
     'release_identifier': 'https://musicbrainz.org/release/c658d3a6-087f-419a-be61-409a76d1a6cc'}},
   'identifier': ['https://musicbrainz.org/recording/6b0b5193-134c-431f-943a-80ff05d34ecf'],
   'title': 'Now He’s Our Father'},
  {'album': 'Save Stereogum: An ’00s Covers Comp',
   'creator': 'Waxahatchee',
   'duration': 188000,
   'extension': {'https://musicbrainz.org/doc/jspf#track': {'artist_identifiers': ['42321e24-42b6-4f08-b0d9-8325ee887a20'],
     'release_identifier': 'https://musicbrainz.org/release/19bd282e-36dc-4a5a-a29d-9289532c647d'}},
   'identifier': ['https://musicbrainz.org/recording/598ea118-8115-4df2-8638-922a61e57212'],
   'title': 'You Said Something'}])

In [8]:
identifiers = []
for track in tracks:
    if len(track['identifier']) > 1:
        raise Exception(f'{track["title"]} by {track["creator"]} has multiple identifiers: {", ".join(track["identifiers"])}')
    identifiers += track["identifier"]
mbids = [x.split("/")[-1] for x in identifiers]
identifiers[:2], mbids[:2]

(['https://musicbrainz.org/recording/6b0b5193-134c-431f-943a-80ff05d34ecf',
  'https://musicbrainz.org/recording/598ea118-8115-4df2-8638-922a61e57212'],
 ['6b0b5193-134c-431f-943a-80ff05d34ecf',
  '598ea118-8115-4df2-8638-922a61e57212'])

In [10]:
all_relations = []
all_isrcs = []
spotify_tracks = []
rate_limit_remaining = 900  # Arbitrary > 0 number
rate_limit_max = 1000 # Slightly larger arbitrary > 0 number
for mbid in mbids:
    time.sleep((rate_limit_max - rate_limit_remaining) / 1000)
    
    # https://musicbrainz.org/doc/MusicBrainz_API#Lookups
    mb_recording = requests.get(
        url=f"https://musicbrainz.org/ws/2/recording/{mbid}",
        params={"fmt": "json", "inc": "isrcs url-rels artists"},
        headers={"User-Agent": "mixtapestudy.com/0.0 ( douglas@builtonbits.com )"}
    )
    handle_error(mb_recording)
    
    recording_json = mb_recording.json()
    rate_limit_remaining = int(mb_recording.headers["X-RateLimit-Remaining"])
    rate_limit_max = int(mb_recording.headers["X-RateLimit-Limit"])
    print(f"{rate_limit_remaining}/{rate_limit_max}")
    relations = recording_json["relations"]
    isrcs = recording_json["isrcs"]
    artist_credit = recording_json["artist-credit"]

    if isrcs:
        query_string = f"isrc:{isrcs[0]}"
    elif relations:
        raise Exception(f"Finally found a relation: {relations}, this never happened before")
    else:
        query_string = f'track:{recording_json["title"]}'
        if artist_credit:
            query_string += " " + " ".join([f'artist:{artist["artist"]["name"]}' for artist in artist_credit])
        
    print(f"query_string: {query_string}")
    
    # https://developer.spotify.com/documentation/web-api/reference/search
    spotify_search = requests.get(
        url="https://api.spotify.com/v1/search",
        params={"type": "track", "q": query_string},
        headers={"Authorization": f"Bearer {spotify_token}"}
    )
    handle_error(spotify_search)

    spotify_json = spotify_search.json()
    # print([t["id"] for t in spotify_json["tracks"]["items"]])
    if spotify_json["tracks"] and spotify_json["tracks"]["items"]:
        spotify_tracks.append(spotify_json["tracks"]["items"][0])
    
    all_relations += relations
    all_isrcs += isrcs
    
    # break

(len(all_relations), len(all_isrcs), len(spotify_tracks)), all_relations[:2], all_isrcs[:2]

687/1200
query_string: isrc:CA2BN1702128
920/1200
query_string: track:You Said Something artist:Waxahatchee
1182/1200
query_string: isrc:USRC12300012
985/1200
query_string: isrc:USC4R2003121
737/1200
query_string: isrc:US38Y0913401
970/1200
query_string: track:5-Leaf-Clover artist:Snail Mail
761/1200
query_string: track:Scott Bass artist:The National
918/1200
query_string: track:Fake Plastic Trees artist:Phoebe Bridgers artist:Arlo Parks
1108/1200
query_string: isrc:USC4R2336302
882/1200
query_string: track:Bread artist:Japanese Breakfast
1162/1200
query_string: track:Calling Long Distance (acoustic version) artist:Matthew and the Atlas
972/1200
query_string: isrc:USWEX1600006
694/1200
query_string: track:Words artist:Storefront Church artist:Phoebe Bridgers
849/1200
query_string: isrc:USUG11700566
1101/1200
query_string: track:Killing the Blues (Live) artist:Manchester Orchestra
874/1200
query_string: track:Last Words of a Shooting Star artist:Mitski
1166/1200
query_string: track:Vibr

Exception: Finally found a relation: [{'type-id': '45d0cbc5-d65b-4e77-bdfd-8a75207cb5c5', 'url': {'id': '946a25f3-6bba-4c45-b234-941be4a0089b', 'resource': 'https://owsey.bandcamp.com/track/bon-iver-re-stacks-owsey-resotone-rework'}, 'source-credit': '', 'ended': False, 'type': 'download for free', 'end': None, 'attributes': [], 'begin': None, 'attribute-ids': {}, 'target-type': 'url', 'direction': 'forward', 'target-credit': '', 'attribute-values': {}}], this never happened before

In [1]:
[
    f'{st["id"]}: {st["name"]}'
    for st in spotify_tracks
]

NameError: name 'spotify_tracks' is not defined

In [15]:
lb_recordings = requests.get(
    url="https://api.listenbrainz.org/1/metadata/recording",
    params={
        "recording_mbids": ",".join(mbids),
        "inc": ""
    },
    headers={"Authorization": f"Bearer {listenbrainz_api_key}"}
)
handle_error(lb_recordings)

print(json.dumps(lb_recordings.json(), indent=4))

{
    "04376d83-606f-494e-82b0-f72c612ae036": {
        "recording": {
            "length": 266000,
            "name": "Mermaid (Topanga demo)",
            "rels": []
        }
    },
    "06f4c620-b0bd-418f-b3c8-19088b7407dc": {
        "recording": {
            "length": 243000,
            "name": "I Couldn't Save You",
            "rels": []
        }
    },
    "096efe0a-43df-423b-b6af-caf1e737d262": {
        "recording": {
            "length": 210413,
            "name": "Trust",
            "rels": [
                {
                    "artist_mbid": "af25706b-0d0f-409a-b5b9-10958acb06f5",
                    "artist_name": "Hayden Cotcher",
                    "instrument": "drums (drum set)",
                    "type": "instrument"
                },
                {
                    "artist_mbid": "869cfae0-b503-4d05-9c04-fc059ba416db",
                    "artist_name": "Christine Moad",
                    "instrument": "bass",
                    "type": "inst